In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import cv2
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import warnings
warnings.filterwarnings('ignore')

from sklearn.metrics import confusion_matrix, classification_report

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, BatchNormalization, Conv2D, Dense, Dropout, Flatten, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

In [ ]:
train_dataset_path = 'Train/'
test_dataset = 'Test/'

In [ ]:
IMG_WIDTH = 150
IMG_HEIGHT = 150
BATCH_SIZE = 32

In [ ]:
train_datagen = ImageDataGenerator(rescale=1.0/255,
                                  zoom_range=0.2,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  fill_mode='nearest')
train_generator = train_datagen.flow_from_directory(train_dataset_path,
                                                   target_size=(IMG_WIDTH, IMG_HEIGHT),
                                                   batch_size=BATCH_SIZE,
                                                   class_mode='categorical',
                                                   shuffle=True)

Found 6144 images belonging to 5 classes.


In [ ]:
validation_datagen = ImageDataGenerator(rescale=1.0/255)
validation_generator = validation_datagen.flow_from_directory(test_dataset,
                                                             target_size=(IMG_WIDTH, IMG_HEIGHT),
                                                             batch_size=BATCH_SIZE,
                                                             class_mode='categorical',
                                                             shuffle=True)

Found 1354 images belonging to 5 classes.


In [ ]:
labels = {value: key for key, value in train_generator.class_indices.items()}

print("Label Mappings for classes present in the training and validation datasets\n")
for key, value in labels.items():
    print(f"{key} : {value}")

Label Mappings for classes present in the training and validation datasets

0 : Cataract
1 : Diabetes
2 : Glaucoma
3 : Normal
4 : Other


In [ ]:
def create_model():
    with tf.device('/gpu:0'):

        model = Sequential([
            Conv2D(filters=128, kernel_size=(5, 5), padding='valid', input_shape=(IMG_WIDTH, IMG_HEIGHT, 3)),
            Activation('relu'),
            MaxPooling2D(pool_size=(2, 2)),
            BatchNormalization(),

            Conv2D(filters=64, kernel_size=(3, 3), padding='valid', kernel_regularizer=l2(0.00005)),
            Activation('relu'),
            MaxPooling2D(pool_size=(2, 2)),
            BatchNormalization(),

            Conv2D(filters=32, kernel_size=(3, 3), padding='valid', kernel_regularizer=l2(0.00005)),
            Activation('relu'),
            MaxPooling2D(pool_size=(2, 2)),
            BatchNormalization(),

            Flatten(),

            Dense(units=256, activation='relu'),
            Dropout(0.5),
            Dense(units=5, activation='softmax')
        ])

        return model

In [ ]:
cnn_model = create_model()

In [ ]:
print(cnn_model.summary())


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 146, 146, 128)     9728      
_________________________________________________________________
activation_6 (Activation)    (None, 146, 146, 128)     0         
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 73, 73, 128)       0         
_________________________________________________________________
batch_normalization_6 (Batch (None, 73, 73, 128)       512       
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 71, 71, 64)        73792     
_________________________________________________________________
activation_7 (Activation)    (None, 71, 71, 64)        0         
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 35, 35, 64)       

In [ ]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=np.sqrt(0.1), patience=5)


In [ ]:
optimizer = Adam(learning_rate=0.001)


In [ ]:
cnn_model.compile(optimizer=optimizer, loss=CategoricalCrossentropy(), metrics=['accuracy'])


In [ ]:
history = cnn_model.fit(train_generator, epochs=10, validation_data=validation_generator,
                       verbose=2,
                       callbacks=[reduce_lr])

Epoch 1/10
192/192 - 68s - loss: 1.3459 - accuracy: 0.4032 - val_loss: 1.3872 - val_accuracy: 0.3656
Epoch 2/10
192/192 - 62s - loss: 1.3482 - accuracy: 0.4041 - val_loss: 1.3531 - val_accuracy: 0.3936
Epoch 3/10
192/192 - 64s - loss: 1.3467 - accuracy: 0.3940 - val_loss: 1.3726 - val_accuracy: 0.3840
Epoch 4/10
192/192 - 65s - loss: 1.3521 - accuracy: 0.3918 - val_loss: 1.3693 - val_accuracy: 0.3900
Epoch 5/10
192/192 - 64s - loss: 1.3353 - accuracy: 0.3991 - val_loss: 1.3853 - val_accuracy: 0.3604
Epoch 6/10
192/192 - 64s - loss: 1.3463 - accuracy: 0.4036 - val_loss: 1.3788 - val_accuracy: 0.3759
Epoch 7/10
192/192 - 64s - loss: 1.3467 - accuracy: 0.3955 - val_loss: 1.3735 - val_accuracy: 0.3752
Epoch 8/10
192/192 - 65s - loss: 1.3375 - accuracy: 0.4131 - val_loss: 1.3367 - val_accuracy: 0.4025
Epoch 9/10
192/192 - 65s - loss: 1.3261 - accuracy: 0.4155 - val_loss: 1.3437 - val_accuracy: 0.4025
Epoch 10/10
192/192 - 65s - loss: 1.3231 - accuracy: 0.4124 - val_loss: 1.3421 - val_accura

In [ ]:
test_datagen = ImageDataGenerator(rescale=1.0/255)

test_generator = test_datagen.flow_from_directory(test_dataset,
                                                 shuffle=False,
                                                 batch_size=BATCH_SIZE,
                                                 target_size = (IMG_WIDTH, IMG_HEIGHT),
                                                 class_mode='categorical')

Found 1354 images belonging to 5 classes.


In [ ]:
predictions = cnn_model.predict(test_generator)


In [ ]:
test_loss, test_accuracy = cnn_model.evaluate(test_generator, batch_size=BATCH_SIZE)


43/43 [==============================] - 4s 83ms/step - loss: 1.3421 - accuracy: 0.4010


In [ ]:
print(f"Test Loss:     {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

Test Loss:     1.3421422243118286
Test Accuracy: 0.4010339677333832


In [ ]:
import pickle
pickle.dump(cnn_model, open('cnn_model.pkl', 'wb'))


#YUCK!!!